In [ ]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from strategy_stepper import *

In [ ]:
def mutate_samples(var : Var, initial_value, n_points=100000, random_walk=False):
    nas_strategy = StepperStrategy()
    mvar = MutableVar(var.type, initial_value, None, var)
    values = []
    steps = []
    unchanged_values = 0
    for i in range(n_points):
        if random_walk:
            if mvar.new_value is not None:
                mvar.value = mvar.new_value
                mvar.step = mvar.new_step
        else:
            mvar.step = None
        mvar.new_value = None
        mvar.new_step = None
    
        if not random_walk:
            mvar.value = initial_value
    
        mutation_state = nas_strategy.MutationState(0, 25, 0.5, 0.5)
    
        nas_strategy.mutate_variable_step_per_parameter(mvar, mutation_state)
        
        if mvar.new_value is not None:
            values.append(mvar.new_value)
        else:
            values.append(mvar.value)
            unchanged_values += 1
        steps.append(mvar.new_step if mvar.new_step != None else mvar.step)
    return values, steps, unchanged_values

def plot_mutated_samples(title, var : Var, initial_value, n_points=100000, random_walk=False, show_walk=False, n_bins=100, n_bins_step=100, step_xlim=None, step_range=None):
    random.seed(1)
    np.random.seed(1)
    
    values, steps, unchanged_values = mutate_samples(var, initial_value, n_points, random_walk)

    print(f"{unchanged_values=} of {n_points}")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7))
    if show_walk:
        ax1.plot(values)
        ax2.plot(steps)
        #     ax2.set_xscale('log')
    else:
        ax1.set_title(("Random Walk of " if random_walk else "Mutation Distribution of ") + title)
        ax1.hist(values, bins=n_bins)
        ax2.set_title("Step Width Distribution")
        ax2.hist(steps, bins=n_bins_step, range=step_range)
        ax2.set_xlim(0, step_xlim)
    plt.show()

plot_mutated_samples("Int Variable with Initial Value 50 and range [0, 100]", Var("test_int", Type.INT, 0, 100), 50, step_xlim=100)

In [ ]:
plot_mutated_samples("Float Variable with Initial Value 50 and range [0, 100]", Var("test_float", Type.FLOAT, 0.0, 100.0), 50.0)
plot_mutated_samples("Float Variable with Initial Value 50 and range [0, 100]", Var("test_float", Type.FLOAT, 0.0, 100.0), 50.0, n_points=1000, random_walk=True)

In [ ]:
plot_mutated_samples("Categorical Variable with Initial Value 'green'", Var("test_cat", Type.CAT, categories=['red', 'green', 'yellow']), 'green', step_xlim=0.6, step_range=(0, 0.6))

In [ ]:
# test of interval_transform
x = np.arange(-200, 300.0, 1)
y = list(map(lambda xi: interval_transform(xi, 0.0, 100.0), x))
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_title("$T_{0,100}(x)$")
ax.plot(x, y)
ax.set_ylim(-200, 300)
ax.set_xlim(-200, 300)
ax.grid()
plt.show()

In [ ]:
random.seed(1)
np.random.seed(1)
values = []
for i in range(100000):
    v = 50 + 100 * random.gauss(0, 0.25)
    v = np.clip(v, 0, 100)
    values.append(v)

fig, ax1 = plt.subplots(figsize=(10, 7))
ax1.set_title("Mutations of a variable with range [0, 100] and σ=0.25, clipped to interval")
ax1.hist(values, bins=100)
ax1.set_xlim(0, 100)
plt.show()

In [ ]:
random.seed(1)
np.random.seed(1)
values = []
for i in range(100000):
    v = 50 + 100 * random.gauss(0, 0.25)
    v = interval_transform(v, 0, 100)
    values.append(v)

fig, ax1 = plt.subplots(figsize=(10, 7))
ax1.set_title("Mutations of a variable with range [0, 100] and σ=0.25, transformed with $T_{0,100}()$")
ax1.hist(values, bins=100)
ax1.set_xlim(0, 100)
plt.show()